In [ ]:
from PIL import Image
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import LabelBinarizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix, classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

In [ ]:
!python --version

In [ ]:
#Leitura do csv raspado para obter os ID's e buscar as imagens
tab = pd.read_csv(r'../input/termographdata/Dados/tabulares.csv', encoding='utf-8', low_memory=False)
caract = pd.read_csv(r'../input/termographdata/Dados/caracteristicas', encoding='utf-8', low_memory=False)
comp = pd.read_csv(r'../input/termographdata/Dados/complementares.csv', encoding='utf-8', low_memory=False)
#Junta dados
df = pd.merge(tab, caract, on='ID')
df = pd.merge(df, comp, on='ID')

In [ ]:
df.head(3)

In [ ]:
#Trata algumas informações de características
df.Estado_civil = df.Estado_civil.str.replace('widow', 'viuva')
df.Estado_civil = df.Estado_civil.str.replace('married', 'casada')
df.Estado_civil = df.Estado_civil.str.replace('single', 'solteira')
df.Estado_civil = df.Estado_civil.str.replace('widow', 'viuva')
df.Estado_civil = df.Estado_civil.str.replace('viuvo', 'viuva')
df.Estado_civil = df.Estado_civil.str.replace('divorced', 'divorciada')
df.Estado_civil = df.Estado_civil.str.replace('casado', 'casada')
df.Estado_civil = df.Estado_civil.str.replace('divorciado', 'divorciada')
df.Estado_civil = df.Estado_civil.str.replace('solteiro', 'solteira')
df.Estado_civil = df.Estado_civil.str.replace('casada ', 'casada')
#
df.Raca = df.Raca.str.replace('black', 'negra')
df.Raca = df.Raca.str.replace('white', 'branca')
df.Raca = df.Raca.str.replace('mulattos', 'mulata')
df.Raca = df.Raca.str.replace('pardo', 'parda')

In [ ]:
lista_drop = ['Records', 'Name', 'Exams', 'Data_registro']
df = df.drop(lista_drop, axis=1)

In [ ]:
#Retira diagnósticos desconhecido
df_unknow = df.query(r"Diagnosis == 'unknown'")
df = df.query(r"Diagnosis != 'unknown'")

Análise Descritiva

In [ ]:
#Idade
fig, ax =  plt.subplots(nrows=1,ncols=2, figsize=(18,6))
fig.suptitle('Idade')
ax[0].hist(df.Age)
ax[0].grid(linestyle=':', linewidth=0.5)
#
ax[1].boxplot(df.Age)
ax[1].grid(linestyle=':', linewidth=0.5)
ax[1].set_xticks([])
plt.show();

In [ ]:
raca = (
    pd.DataFrame(df.groupby('Raca')['ID'].count())
    .reset_index()
    .rename(columns={'ID':'Quantidade'})
)
est = (
    pd.DataFrame(df.groupby('Estado_civil')['ID'].count())
    .reset_index()
    .rename(columns={'ID':'Quantidade'})
)
hab = (
    pd.DataFrame(df.groupby('Hábitos Alimentares')['ID'].count())
    .reset_index()
    .rename(columns={'ID':'Quantidade'})
)
rad = (
    pd.DataFrame(df.groupby('Radioterapia')['ID'].count())
    .reset_index()
    .rename(columns={'ID':'Quantidade'})
)
sin = (
    pd.DataFrame(df.groupby('Sinais')['ID'].count())
    .reset_index()
    .rename(columns={'ID':'Quantidade'})
)
plas = (
    pd.DataFrame(df.groupby('Cirurgia Plástica')['ID'].count())
    .reset_index()
    .rename(columns={'ID':'Quantidade'})
)

In [ ]:
fig, ax =  plt.subplots(nrows=2,ncols=3, figsize=(20,12))
plt.subplots_adjust(wspace= 0.2, hspace= 0.5)

plt.setp(ax[0][0].get_xticklabels(), Rotation=45,ha='right',fontsize=15, color = 'w')
plt.setp(ax[0][0].get_yticklabels(),fontsize=15, color = 'w')
ax[0][0].bar(raca.Raca, raca.Quantidade)
ax[0][0].grid(linestyle=':', linewidth=0.5)
ax[0][0].set_title('Raça', color='w')
#ax[0][0].title.set_text('Raça')

#
plt.setp(ax[0][1].get_xticklabels(), Rotation=45,ha='right',fontsize=15, color = 'w')
plt.setp(ax[0][1].get_yticklabels(),fontsize=15, color = 'w')
ax[0][1].bar(est.Estado_civil, est.Quantidade )
ax[0][1].grid(linestyle=':', linewidth=0.5)
#ax[0][1].title.set_text('Estado Civil')
ax[0][1].set_title('Estado Civil', color='w')

#
plt.setp(ax[0][2].get_xticklabels(), Rotation=45,ha='right',fontsize = 15, color = 'w')
plt.setp(ax[0][2].get_yticklabels(),fontsize=15, color = 'w')
ax[0][2].bar(hab['Hábitos Alimentares'], hab.Quantidade )
ax[0][2].grid(linestyle=':', linewidth=0.5)
#ax[0][2].title.set_text('Hábitos alimentares')
ax[0][2].set_title('Hábitos alimentares', color='w')
#
plt.setp(ax[1][0].get_xticklabels(),fontsize = 15, color = 'w')
plt.setp(ax[1][0].get_yticklabels(),fontsize=14, color = 'w')
ax[1][0].bar(rad['Radioterapia'], rad.Quantidade )
ax[1][0].grid(linestyle=':', linewidth=0.5)
#ax[1][0].title.set_text('Radioterapia')
ax[1][0].set_title('Radioterapia', color='w')
#
plt.setp(ax[1][1].get_xticklabels(),fontsize = 15, color = 'w')
plt.setp(ax[1][1].get_yticklabels(),fontsize=15, color = 'w')
ax[1][1].bar(sin['Sinais'], sin.Quantidade )
ax[1][1].grid(linestyle=':', linewidth=0.5)
#ax[1][1].title.set_text('Marcas / Sinais')
ax[1][1].set_title('Marcas / Sinais', color='w')
#
plt.setp(ax[1][2].get_xticklabels(),fontsize = 15, color = 'w')
plt.setp(ax[1][2].get_yticklabels(),fontsize=15, color = 'w')
ax[1][2].bar(plas['Cirurgia Plástica'], plas.Quantidade )
ax[1][2].grid(linestyle=':', linewidth=0.5)
#ax[1][2].title.set_text('Cirurgia Plástica')
ax[1][2].set_title('Cirurgia Plástica', color='w')
#
plt.savefig('Desritiva.png', dpi=500, bbox_inches='tight', pad_inches=0.5)
plt.show();

In [ ]:
df['Menarca'] = df['Menarca'].astype(int)
#IMenarca
fig, ax =  plt.subplots(nrows=1,ncols=2, figsize=(12,4))
fig.suptitle('Menarca')
ax[0].hist(df.Menarca)
ax[0].grid(linestyle=':', linewidth=0.5)
#
ax[1].boxplot(df.Menarca)
ax[1].grid(linestyle=':', linewidth=0.5)
ax[1].set_xticks([])
plt.show();

In [ ]:
t = df.Diagnosis.value_counts()[0]
df.shape[0]

In [ ]:
df.Diagnosis.value_counts()

In [ ]:
dia = (
    pd.DataFrame(df.groupby('Diagnosis')['ID'].count())
    .reset_index()
    .rename(columns={'ID':'Quantidade'})
)
plt.rcParams["figure.figsize"] = (6,4)
    
perc_healt = round(df.Diagnosis.value_counts()[0] / df.shape[0] * 100,3)
perc_healt = str(perc_healt) + '%'
#
perc_sick = round(df.Diagnosis.value_counts()[1] / df.shape[0] * 100,3)
perc_sick = str(perc_sick) + '%'    
plt.title('Diagnóstico')
plt.bar(dia['Diagnosis'], dia.Quantidade )
plt.grid(linestyle=':', linewidth=0.5)
plt.annotate(perc_healt, 
              xy=(-0.11, 98))
plt.annotate(perc_sick, 
              xy=(0.89, 45))

plt.show();

In [ ]:
#Converte imagem para array
def convert_image(image_to_convert):
  # convert to RGB, if needed
  image_to_convert = image_to_convert.convert('RGB')
  image_to_convert = image_to_convert.resize((240,240))
  # convert to array
  return np.asarray(image_to_convert)

In [ ]:
def select_images(patient):
  #Define array para receber imagens da paciente
  patient_images_array = []
  #Atribui caminho base para pastas de imagens
  images_directory = '../input/termographdata/Dados/Imagens/'
  #De acordo com o id da paciente, monta nome e pesquisa pela pasta dela
  patient_directory = (images_directory + 'PACIENTE_' + str(patient).zfill(4))
  #Veririca a quantidade de imagens dentro da pasta da paciente
  list_images = os.listdir(patient_directory) 
  for i, image in enumerate(list_images):
    #instancia imagem
    termography = Image.open(patient_directory + '/' + image)
    #Converte imagem
    converted_image = convert_image(termography)
    patient_images_array.append(converted_image)
  print('Paciente: ' + str(patient).zfill(4) + ' processado')
  return(patient_images_array)

In [ ]:
#Itera sobre o dataframe e inclui nova coluna com todas imagens em array
df['Imagens'] = df.ID.apply(select_images)

In [ ]:
#Conta diagnosis para saber quantos temos de cada
df.Diagnosis.value_counts()

In [ ]:
#Binariza Diagnosis
lb = LabelBinarizer()
df['Diagnosis'] = to_categorical(lb.fit_transform(df['Diagnosis']))

In [ ]:
#Conta diagnosis para saber quantos temos de cada
df.Diagnosis.value_counts()

In [ ]:
#Binariza outros campos binários
#
df.Radioterapia = to_categorical(lb.fit_transform(df.Radioterapia))
#
df['Cirurgia Plástica'] = to_categorical(lb.fit_transform(df['Cirurgia Plástica']))
#
df['Sinais'] = to_categorical(lb.fit_transform(df['Sinais']))

In [ ]:
#Trata demais variáveis categóricas não binárias
le = preprocessing.LabelEncoder()
df.Estado_civil = le.fit_transform(df.Estado_civil)
df.Raca = le.fit_transform(df.Raca)
df['Hábitos Alimentares'] = le.fit_transform(df['Hábitos Alimentares'])
df.Raca = le.fit_transform(df.Raca)

In [ ]:
#Separa dependentes e independentes
x_mach = df.drop(columns=['ID', 'Diagnosis', 'Imagens'])
y_mach = df.Diagnosis
#Split em treino e teste para machine learning
x_train_mach, x_test_mach, y_train_mach ,y_test_mach = train_test_split(x_mach, y_mach, test_size=0.25,random_state=42)
#Para deep será feito depois

Começa treinamento dos modelos de ml

In [ ]:
#instanciando modelos
dt = DecisionTreeClassifier(criterion='entropy', max_leaf_nodes=4, max_depth=6, min_samples_leaf=3, random_state=42)
lr = LogisticRegression(max_iter=500, random_state=42,fit_intercept=False,solver='liblinear')
rf = RandomForestClassifier(criterion='entropy', max_leaf_nodes=4, max_depth=6, min_samples_leaf=3, random_state=42)
xg = XGBClassifier(n_estimators=10,use_label_encoder=False)
rg = RidgeClassifier(random_state = 42, alpha=20, fit_intercept=False)
#
models_ml = [dt,lr,rf,xg,rg]

#Treinando
for model in models_ml:
    model.fit(x_train_mach, y_train_mach)

In [ ]:
#Find best com model_score
def find_best_class(x,y,cv):
    #instanciando 3 modelos
    dt = DecisionTreeClassifier(criterion='entropy', random_state=42)
    lr = LogisticRegression(max_iter=400, random_state=42)
    rf = RandomForestClassifier(criterion='entropy', random_state=42)
    xg = XGBClassifier(learning_rate = 0.1, max_depth = 5, alpha = 10, n_estimators = 10,use_label_encoder=False)
    rg = RidgeClassifier(alpha=2)
    ## gerando lista de modelos
    lst_models = [dt,lr,rf,xg,rg]

    ## gerando dicionário de scores
    models_score = {}
    #
    ## lista de valores de métricas
    plot_met = []
    ## iterando sobre modelos e executando a validação
    for i ,model in enumerate(lst_models):
        models_score[i] = cross_val_score(model,x,y,cv=cv).mean()
        plot_met.append(models_score[i])
#    
    best_model = min(models_score,key=models_score.get)
#
    print('-- Comparando modelos --')
#    
    if best_model == 0:
        print('Melhor modelo - DecisionTree: %f' %(models_score[0]))
    elif best_model == 1:
        print('Melhor modelo - LogisticRegression: %f' %(models_score[1]))
    elif best_model == 2:
        print('Melhor modelo - RandomForestClassifier: %f' %(models_score[2]))
    elif best_model == 3:
        print('Melhor modelo - XGboost: %f' %(models_score[3]))
    elif best_model == 4:
        print('Melhor modelo - RidgeClassifier: %f' %(models_score[4]))
#
    plt.bar(['dt','lr','rf','xg', 'rg'], plot_met)
    plt.show();

In [ ]:
find_best_class(x_train_mach,y_train_mach,8)

In [ ]:
#Find best com test_accuracy
def find_best_class_acc(x,y):
  scoring=['accuracy',
         'average_precision',
         'f1',
         'precision',
         'recall',
         'roc_auc']
  #instanciando 3 modelos
  dt = DecisionTreeClassifier(criterion='entropy', max_leaf_nodes=4, max_depth=6, min_samples_leaf=3, random_state=42)
  lr = LogisticRegression(max_iter=500, random_state=42,fit_intercept=False,solver='liblinear')
  rf = RandomForestClassifier(criterion='entropy', max_leaf_nodes=4, max_depth=6, min_samples_leaf=3, random_state=42)
  xg = XGBClassifier(n_estimators=10)
  rg = RidgeClassifier(random_state = 42, alpha=20, fit_intercept=False)
  ## gerando lista de modelos
  lst_models = [dt,lr,rf,xg,rg]
  ## gerando dicionário de scores
  models_score = []
  #
  ## lista de valores de métricas
  plot_met = []
  ## iterando sobre modelos e executando a validação
  for i ,model in enumerate(lst_models):
    aux = cross_validate(model, x, y, return_train_score=False, scoring=scoring)
    aux = aux['test_accuracy'].mean()
    models_score.append(aux)
    plot_met.append(models_score[i])
#    
  best_model_value = max(models_score)
  best_model_index = (models_score.index(best_model_value))

#
  print('-- Comparando modelos --')
#    
  if best_model_index == 0:
      print('Melhor modelo - DecisionTree: %f' %(models_score[0]))
  elif best_model_index == 1:
      print('Melhor modelo - LogisticRegression: %f' %(models_score[1]))
  elif best_model_index == 2:
      print('Melhor modelo - RandomForestClassifier: %f' %(models_score[2]))
  elif best_model_index == 3:
      print('Melhor modelo - XGboost: %f' %(models_score[3]))
  elif best_model_index == 4:
      print('Melhor modelo - RidgeClassifier: %f' %(models_score[4]))
#
  plt.bar(['dt','lr','rf','xg', 'rg'], plot_met)
  print('*******************************************')
  print_mdl = ['dt','lr','rf','xg','rg']
  for i, mdl in enumerate(print_mdl):
    print(mdl,':',  models_score[i])
  plt.show();

In [ ]:
#Executa find best com test_accuracy para dados de treino
find_best_class_acc(x_train_mach,y_train_mach)

In [ ]:
##Executa find best com test_accuracy para dados de teste
find_best_class_acc(x_test_mach,y_test_mach)

In [ ]:
pred = dt.predict(x_test_mach)
cm = confusion_matrix(y_test_mach, pred)
total = sum(sum(cm))
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
#
fig, ax = plot_confusion_matrix(conf_mat=cm ,  figsize=(4, 4))
plt.title('Decision Tree')
#
plt.show()
print('**********************************')
print('Acurácia ---------------> %f' %acc)
print('Sensitividade ----------> %f' %sensitivity)
print('Especificidade ---------> %f' %specificity)
print('**********************************')
print('\n')
####
####
pred = lr.predict(x_test_mach)
cm = confusion_matrix(y_test_mach, pred)
total = sum(sum(cm))
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
#
fig, ax = plot_confusion_matrix(conf_mat=cm ,  figsize=(4, 4))
plt.title('Linear Regression')
#
plt.show()
print('**********************************')
print('Acurácia ---------------> %f' %acc)
print('Sensitividade ----------> %f' %sensitivity)
print('Especificidade ---------> %f' %specificity)
print('**********************************')
print('\n')
####
####
pred = rf.predict(x_test_mach)
cm = confusion_matrix(y_test_mach, pred)
total = sum(sum(cm))
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
#
fig, ax = plot_confusion_matrix(conf_mat=cm ,  figsize=(4, 4))
plt.title('Random Forest')
#
plt.show()
print('**********************************')
print('Acurácia ---------------> %f' %acc)
print('Sensitividade ----------> %f' %sensitivity)
print('Especificidade ---------> %f' %specificity)
print('**********************************')
print('\n')
####
####
pred = xg.predict(x_test_mach)
cm = confusion_matrix(y_test_mach, pred)
total = sum(sum(cm))
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
#
fig, ax = plot_confusion_matrix(conf_mat=cm ,  figsize=(4, 4))
plt.title('XGBosst')
#
plt.show()
print('**********************************')
print('Acurácia ---------------> %f' %acc)
print('Sensitividade ----------> %f' %sensitivity)
print('Especificidade ---------> %f' %specificity)
print('**********************************')
print('\n')
####
####
pred = rg.predict(x_test_mach)
cm = confusion_matrix(y_test_mach, pred)
total = sum(sum(cm))
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
#
fig, ax = plot_confusion_matrix(conf_mat=cm ,  figsize=(4, 4))
plt.title('Ridge Classifier')
#
plt.show()
print('**********************************')
print('Acurácia ---------------> %f' %acc)
print('Sensitividade ----------> %f' %sensitivity)
print('Especificidade ---------> %f' %specificity)
print('**********************************')
print('\n')

In [ ]:
for model in models_ml:
    pred = model.predict(x_test_mach)
    cm = confusion_matrix(y_test_mach, pred)
    total = sum(sum(cm))
    acc = (cm[0, 0] + cm[1, 1]) / total
    sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
#
    fig, ax = plot_confusion_matrix(conf_mat=cm ,  figsize=(4, 4))
    plt.title('Decision Tree')
#
plt.show()
print('**********************************')
print('Acurácia ---------------> %f' %acc)
print('Sensitividade ----------> %f' %sensitivity)
print('Especificidade ---------> %f'  %specificity)
print('**********************************')
print('\n')
####
####
pred = lr.predict(x_test_mach)
cm = confusion_matrix(y_test_mach, pred)
total = sum(sum(cm))
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
#
fig, ax = plot_confusion_matrix(conf_mat=cm ,  figsize=(4, 4))
plt.title('Linear Regression')
#
plt.show()
print('**********************************')
print('Acurácia ---------------> %f' %acc)
print('Sensitividade ----------> %f' %sensitivity)
print('Especificidade ---------> %f' %specificity)
print('**********************************')
print('\n')
####
####
pred = rf.predict(x_test_mach)
cm = confusion_matrix(y_test_mach, pred)
total = sum(sum(cm))
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
#
fig, ax = plot_confusion_matrix(conf_mat=cm ,  figsize=(4, 4))
plt.title('Random Forest')
#
plt.show()
print('**********************************')
print('Acurácia ---------------> %f' %acc)
print('Sensitividade ----------> %f' %sensitivity)
print('Especificidade ---------> %f' %specificity)
print('**********************************')
print('\n')
####
####
pred = xg.predict(x_test_mach)
cm = confusion_matrix(y_test_mach, pred)
total = sum(sum(cm))
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
#
fig, ax = plot_confusion_matrix(conf_mat=cm ,  figsize=(4, 4))
plt.title('XGBosst')
#
plt.show()
print('**********************************')
print('Acurácia ---------------> %f' %acc)
print('Sensitividade ----------> %f' %sensitivity)
print('Especificidade ---------> %f' %specificity)
print('**********************************')
print('\n')
####
####
pred = rg.predict(x_test_mach)
cm = confusion_matrix(y_test_mach, pred)
total = sum(sum(cm))
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
#
fig, ax = plot_confusion_matrix(conf_mat=cm ,  figsize=(4, 4))
plt.title('Ridge Classifier')
#
plt.show()
print('**********************************')
print('Acurácia ---------------> %f' %acc)
print('Sensitividade ----------> %f' %sensitivity)
print('Especificidade ---------> %f' %specificity)
print('**********************************')
print('\n')

In [ ]:
## classification report
print('******************** Decision Tree ********************')
print(classification_report(y_train_mach,dt.predict(x_train_mach)))
#
## classification report
print('******************** Decision Tree ********************')
print(classification_report(y_train_mach,dt.predict(x_train_mach)))

In [ ]:
lst_models = [dt,lr,rf,xg,rg]
print_mdl  = ['Decision Tree' ,'LogisticRegression', 'RandomForest', 'XGBoost', 'RidgeClassifier']

for i, model in enumerate(lst_models):
  print('*******************', print_mdl[i], '********************')
  print(classification_report(y_train_mach, model.predict(x_train_mach)))

In [ ]:
predicts_mach = y_test_mach

In [ ]:
predicts_mach = pd.DataFrame(list(zip(predicts_mach)), columns = ['Diagnosis'])

In [ ]:
predicts_mach['pred_dt'] = dt.predict(x_test_mach)
predicts_mach['pred_lr'] = lr.predict(x_test_mach)
predicts_mach['pred_rf'] = rf.predict(x_test_mach)
predicts_mach['pred_xg'] = xg.predict(x_test_mach)
predicts_mach['pred_rg'] = rg.predict(x_test_mach)

In [ ]:
predicts_mach.head(5)

In [ ]:
#Visualiza
fig, ax =  plt.subplots(nrows=1,ncols=5, figsize=(22,3))

ax[0].scatter(predicts_mach.index,predicts_mach.Diagnosis, color='red', s=40, label='Diagnosis')
ax[0].scatter(predicts_mach.index,predicts_mach.pred_dt, color='blue', s=20, label='Decision Tree')
ax[0].legend(fontsize=8, loc='center left')
#
ax[1].scatter(predicts_mach.index,predicts_mach.Diagnosis, color='red', s=40, label='Diagnosis')
ax[1].scatter(predicts_mach.index,predicts_mach.pred_lr, color='green', s=20, label='Linear Regression')
ax[1].legend(fontsize=8, loc='center left')
#
ax[2].scatter(predicts_mach.index,predicts_mach.Diagnosis, color='red', s=40, label='Diagnosis')
ax[2].scatter(predicts_mach.index,predicts_mach.pred_rf, color='yellow', s=20, label='Randon Forest')
ax[2].legend(fontsize=8, loc='center left')
#
ax[3].scatter(predicts_mach.index,predicts_mach.Diagnosis, color='red', s=40, label='Diagnosis')
ax[3].scatter(predicts_mach.index,predicts_mach.pred_xg, color='lime', s=20, label='XGboost')
ax[3].legend(fontsize=8, loc='center left')
#
ax[4].scatter(predicts_mach.index,predicts_mach.Diagnosis, color='red', s=40, label='Diagnosis')
ax[4].scatter(predicts_mach.index,predicts_mach.pred_rg, color='orange', s=20, label='Ridge')
ax[4].legend(fontsize=8, loc='center left')
#
plt.show();

In [ ]:
import gc
#
del tab
del caract
del comp
del lista_drop
#del patient_images_array
#del images_directory
#del patient_directory
#del list_images
#del termography
#del converted_image
#del patient_images_array
del le
del dt
del lr
del rf
del xg
del rg
del lst_models
#del plot_met
#del aux
#del best_model_value
#del best_model_index
#del print_mdl
del predicts_mach
gc.collect()

In [ ]:
#Prepara arquivo de imagens - disvincula imagens das pacientes
#gera dataframe somente com Diagnosis e imagem
def separa_df(dataframe):
  IMG = []
  DIA = []
  for i in range(dataframe.shape[0]):
    for imagem in (dataframe.iloc[i,1]):
      DIA.append(dataframe.iloc[i,0])
 #    IMG.append(np.asarray(imagem))
      IMG.append(imagem)
  return(DIA, IMG) 

In [ ]:
#Novo dataframe só com imagens e diagnósticos
df_imagens =df[['Diagnosis','Imagens']]
#
#Aciona função para desvincular imagem da paciente com o diagnóstico específico
diag, imagem = separa_df(df_imagens)
diag_n = diag
#
#Transforma as listas de imagens e diagnósticos em array para o keras
imagem = np.array(imagem)          ## convertendo de lista para array
diag = np.array(diag)              ## convertendo de lista para array

In [ ]:
#Binariza e transforma os dados de diagnósticos
diag = to_categorical(lb.fit_transform(diag))

In [ ]:
#Definindo hiperparâmetros do modelo
batch_size   = 32
#Tamanho da imagem
input_shape  = (240, 240, 3)
#
alpha        = 1e-4
epoch        = 10
random_state = 42

In [ ]:
'''
CALLBACKS
Callback são funções que auxiliam o treinamento do modelo usando o Keras. As funções utilizadas serão:

ModelCheckpoint
ReduceLROnPlateau
'''
###
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
###

In [ ]:
'''
ModelCheckpoint
Salva o modelo para cada época, para treinar o modelo sem preocupações com possíveis problemas, como travamento da máquina.

filepath: Local onde o melhor modelo será salvo
monitor: Métrica a ser monitorada para determinar se o modelo será salvo
verbose: (1)mostra na barra de progresso  - (0)não mostra barra de progresso
save_best_only: Salvar somente o melhor modelo
mode: Monitorar o 'acc' o valor aqui vai ser 'max'.
'''
###
filepath='transferlearning_weights.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
###

In [ ]:
'''
ReduceLROnPlateau
Auxilia a reduzir a taxa de aprendizado pelo fator (factor) caso não ocorra a mudança no loss.
factor: fator de redução caso estejamos em um platô
min_delta: valor mínimo da perda
patience: só altere pelo fator após se repitir por 'patience' vezes.
'''
lr_reduce = ReduceLROnPlateau(monitor='acc', factor=0.1, min_delta=alpha, patience=5, verbose=1)

In [ ]:
#Array de callbacks
callbacks = [checkpoint, lr_reduce]

In [ ]:
#Separa em treino e teste
(x_train_deep, x_test_deep, y_train_deep, y_test_deep) = train_test_split(imagem, diag, test_size=0.30, stratify=diag, random_state=42)


In [ ]:
#Data augmentation
train_datagen = ImageDataGenerator(
        rotation_range=10,
        zoom_range=0.1)
train_datagen.fit(x_train_deep)
data_aug = train_datagen.flow(x_train_deep, y_train_deep, batch_size=batch_size)

TRANSFER LEARNING

In [ ]:
conv_base = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)
conv_base.summary()

In [ ]:
#Retreinando parte da VGG19
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
  if layer.name == 'block5_conv1':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False
#
conv_base.summary()

In [ ]:
#Instanciando modelo
model = models.Sequential()
model.add(conv_base)
model.add(layers.GlobalAveragePooling2D()) #Pooling redução de resolução
model.add(layers.BatchNormalization()) #Adiciona normalização
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.4))
model.add(layers.Dense(2, activation='softmax'))
#
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])

In [ ]:
history = model.fit(data_aug,
                    steps_per_epoch=len(x_train_deep)// batch_size, # parte inteira da divisão
                    validation_data=(x_test_deep, y_test_deep),
                    validation_steps=len(x_test_deep) // batch_size,# parte inteira da divisão
                    callbacks=callbacks,
                    epochs=epoch)

Plotando os resultados

In [ ]:
#Gerando matriz de confusão para deep learning
#Realiza predição
pred = model.predict(x_test_deep)
#Transforma os dados
pred = np.argmax(pred,axis = 1) 
y_true = np.argmax(y_test_deep,axis = 1)

In [ ]:
#Plota
cm = confusion_matrix(y_true, pred)
total = sum(sum(cm))
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
#
fig, ax = plot_confusion_matrix(conf_mat=cm ,  figsize=(7, 7))
print('Acurácia      : %f' %acc)
print('Sensitividade : %f' %sensitivity)
print('Especificidade: %f' %specificity)
#
plt.show();

In [ ]:
diag_y_test = []
for value in y_test_deep:
  diag_y_test.append(value[0])

In [ ]:
list_pred = []
for value in pred:
  list_pred.append(value)

In [ ]:
to_plot = pd.DataFrame(list(zip(diag_y_test,list_pred)), columns = ['Diagnosis','pred'])
to_plot['Diagnosis'] = to_plot['Diagnosis'].astype(int)

In [ ]:
plt.rcParams["figure.figsize"] = (20,0.8)
plt.xlim([0,max(to_plot.index) + 1])
plt.scatter(to_plot.index, to_plot.Diagnosis, color='red', s=30, label='Diagnosis')
plt.scatter(to_plot.index, to_plot.pred, color='blue', s=5, label='Predict')
#ax[0].scatter(predicts_mach.index,predicts_mach.pred_dt, color='blue', s=20, label='Decision Tree')
#ax[0].legend(fontsize=8, loc='center left')

In [ ]:
import sklearn.metrics as metrics
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

(a, b, c, val_to_auc) = train_test_split(imagem, diag_n, test_size=0.30, random_state=42)

fpr, tpr, _ = metrics.roc_curve(diag_n,  pred)
auc = metrics.roc_auc_score(diag_n, pred)
plt.plot(fpr,tpr,label=" Acurrácia, auc="+str(auc))
plt.legend(loc=4)
plt.show()

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()